In [11]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

In [12]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [65]:
import minsearch
import json
import os
from openai import OpenAI

In [52]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [53]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [54]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [55]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
    """.strip()

    context = ""
        
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role":"user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [58]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [59]:
es_client.info()

ObjectApiResponse({'name': '826638b85242', 'cluster_name': 'docker-cluster', 'cluster_uuid': '1-y1rY5rQce44Hfz4WV0OQ', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [60]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/wDKddLzSQyWhSVxzxNqoPQ] already exists')

In [20]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:05<00:00, 176.92it/s]


In [61]:
def es_response(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    return es_client.search(index=index_name, body=search_query)

def elastic_search(query):
    response = es_response(query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [62]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [63]:
query = 'How to run kafka?'

In [75]:
answers = rag(query)

In [76]:
answers

"To run Kafka for a Java producer, you can execute the following command in your project directory:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor Python Kafka operations, it's recommended to create and use a virtual environment. First, set up the virtual environment and install the necessary packages:\n\n```bash\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\n```\n\nEvery time you want to run the Python files within this environment, activate the virtual environment:\n\n```bash\nsource env/bin/activate\n```\n\nAnd deactivate it when done:\n\n```bash\ndeactivate\n```\n\nOn Windows, activate the environment with:\n\n```bash\nenv\\Scripts\\activate\n```"